In [36]:
import os
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModel, pipeline, AutoModelForSequenceClassification
import numpy as np
from datasets import Dataset, load_metric
import pandas as pd


In [55]:
df = pd.read_pickle('../../../Files/Submissions/train/submission_train_sm.pickle')



# tokenizer = AutoTokenizer.from_pretrained("j-hartmann/sentiment-roberta-large-english-3-classes")

# model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/sentiment-roberta-large-english-3-classes")

#, warmup = 600, max_sequence_length=128, training_rate=1e-5,

df['text'] = df['title']
df['label'] = df['label'].fillna(0)
df['label'] = df['label'].astype(int)
df.loc[df["label"] == -1, "label"] = 2
# df['labels'] = df['label']
df = df[['text', 'label']]
# df = df[df['labels'] != '0']
# df = df[df['labels'].notna()]
# df.loc[df["labels"] == -1, "labels"] = 0


model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = Dataset.from_pandas(df, preserve_index=False)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



tokenized_dataset = dataset.map(tokenize_function, batched=True)

dataset_splitted = tokenized_dataset.shuffle(1337).train_test_split(0.1)

dataset_splitted

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/jakobschlierf/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/r

  0%|          | 0/64 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 57523
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6392
    })
})

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/jakobschlierf/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tr

In [56]:


for name, param in model.named_parameters():
    if name in ['classifier.weight', 'classifier.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)




training_args = TrainingArguments(
    load_best_model_at_end=True,
    output_dir = '../../Files/models/bert_base_cased_model/',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=32, 
    evaluation_strategy='epoch',
    logging_dir='../../Files/logs/', 
    save_strategy = "epoch",
    save_steps=10_000, save_total_limit=2, )



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_splitted['train'],
    eval_dataset=dataset_splitted['test'],
    compute_metrics=compute_metrics,
)


trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 57523
  Num Epochs = 5
  Instantaneous batch size

  0%|          | 0/8990 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [48]:
trainer.train_dataset[0]["label"]